In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
paths = [ 
          [(7,40), (12, 40), (13,45), (21,45), (24,50), (30,50), (35,55), (40,55), (45,55), (50,55), (55, 55), (60,55), (65,55), (70,58), (75,56), (80,57), (85,55), (90,58), (100,58)],
          [(5,60), (10, 60), (15,60), (20,60), (25,60), (30,60), (35,60), (40,60), (45,60), (50,60), (55, 60), (60,60), (65,60), (70,60), (75,60), (80,60), (85,60), (90,60), (100,60)],
          [(4,90), (11, 90), (16,85), (22,85), (26,85), (30,85), (35,83), (40,78), (45,78), (50,75), (55, 76), (60,72), (65,68), (70,68), (75,67), (80,65), (85,63), (90,62), (100,63)],
        ]
svg = '<svg x="0" y="0" viewbox="0 0 120 120" width="240" height="240"><rect x="0" y="0" width="120" height="120" fill="#ffffff"/>'
for path in paths:
    for i in range(len(path)-1):
        pt0,pt1 = path[i], path[i+1]
        svg += f'<line x1="{pt0[0]}" y1="{pt0[1]}" x2="{pt1[0]}" y2="{pt1[1]}" stroke="#000000" stroke-width="0.2" />'
for path in paths:
    for pt in path:
        svg += f'<circle cx="{pt[0]}" cy="{pt[1]}" r="1" fill="#000000" />'
svg += '</svg>'
svg2 = '<svg x="0" y="0" viewbox="0 0 120 120" width="240" height="240"><rect x="0" y="0" width="120" height="120" fill="#ffffff"/>'
for path in paths:
    _path_ = rt.expandSegmentsIntoPiecewiseCurvedParts(path, amp=1.0, ampends=2.0)
    for i in range(len(_path_)-1):
        pt0,pt1 = _path_[i], _path_[i+1]
        svg2 += f'<line x1="{pt0[0]}" y1="{pt0[1]}" x2="{pt1[0]}" y2="{pt1[1]}" stroke="#000000" stroke-width="0.2" />'
svg2 += '</svg>'
rt.tile([svg,svg2])

In [ ]:
# Bounds
x_min = x_max = paths[0][0][0]
y_min = y_max = paths[0][0][1]
for path in paths:
    for pt in path:
        x_min, x_max = min(x_min,pt[0]), max(x_max,pt[0])
        y_min, y_max = min(x_min,pt[1]), max(x_max,pt[1])
# QuadTree Creation & Population
qt = rt.xyQuadTree((x_min,y_min,x_max,y_max))
path_to_pt = {}
for path_i in range(len(paths)):
    path_to_pt[path_i] = {}
    path = paths[path_i]
    for pt_i in range(len(path)):
        pt = path[pt_i]
        path_to_pt[path_i][pt_i] = pt
        pt_ref = (pt[0], pt[1], path_i, pt_i)
        qt.add([pt_ref])
# Closest other path point to each point in the structure
closests    = {} # pt_ref to all of the closest (other) path points
orig_to_sim = {} # original to the simulated point
for path_i in range(len(paths)):
    path = paths[path_i]
    for pt_i in range(len(path)):
        pt                  = path[pt_i]
        pt_ref              = (pt[0], pt[1], path_i, pt_i)
        closests[pt_ref]    = set()
        orig_to_sim[pt_ref] = pt
        cl                  = qt.closest(pt, n=10)
        seen = set([path_i])
        for o in cl:
            if o[1][2] not in seen:
                seen.add(o[1][2])
                closests[pt_ref].add(o)
help(rt.unitVector)

In [ ]:
g         = 0.1 # gravitational constant (made up)
sim_steps = 1
for iter in range(sim_steps):
    # pull towards the closest point based on gravity formula
    adj = {}
    for pt_ref in orig_to_sim.keys():
        if pt_ref[3] == 0 or pt_ref[3] == (len(paths[pt_ref[2]])-1): # leave the first and last points cemented in...
            continue
        cur = orig_to_sim[pt_ref]
        x_inc, y_inc = 0, 0
        for closest_ref in closests[pt_ref]:
            closest = orig_to_sim[closest_ref[1]]
            s = ((cur[0],cur[1]),(closest[0],closest[1]))
            v = rt.unitVector(s)
            l = rt.segmentLength(s)
            if l > 0.001:
                x_inc += (g*v[0])/(l**2)
                y_inc += (g*v[1])/(l**2)
        adj[pt_ref] = (x_inc,y_inc)
    for pt_ref in adj.keys():
        orig_to_sim[pt_ref] = (orig_to_sim[pt_ref][0] + adj[pt_ref][0], orig_to_sim[pt_ref][0] + adj[pt_ref][0])
    # pull back towards path-related neighbors
    adj = {}
    for path_i in range(len(paths)):
        path = paths[path_i]
        for pt_i in range(1,len(path)-1):
            pt      = path[pt_i]
            pt_ref  = (pt[0], pt[1], path_i, pt_i)
            pt_sim  = orig_to_sim[pt_ref]
            pt0     = path[pt_i-1]
            pt0_ref = (pt0[0], pt0[1], path_i, pt_i-1)
            pt0_sim = orig_to_sim[pt0_ref]
            pt1     = path[pt_i+1]
            pt1_ref = (pt1[0], pt1[1], path_i, pt_i+1)
            pt1_sim = orig_to_sim[pt1_ref]
            pt_avg  = ((pt0_sim[0] + pt1_sim[0])/2, (pt0_sim[1] + pt1_sim[1])/2)
            s       = (pt_sim,pt_avg)
            v       = rt.unitVector(s)
            l       = rt.segmentLength(s)
            adj[pt_ref] = (l*v[0],l*v[1])
    for pt_ref in adj.keys():
        orig_to_sim[pt_ref] = (orig_to_sim[pt_ref][0] + adj[pt_ref][0], orig_to_sim[pt_ref][0] + adj[pt_ref][0])

svg = '<svg x="0" y="0" viewbox="0 0 120 120" width="240" height="240"><rect x="0" y="0" width="120" height="120" fill="#ffffff"/>'
for path_i in range(len(paths)):
    path = paths[path_i]
    for pt_i in range(len(path)):
        pt = path[pt_i]
        pt_ref = (pt[0],pt[1],path_i,pt_i)
        pt_sim = orig_to_sim[pt_ref]
        svg += f'<line x1="{pt[0]}" y1="{pt[1]}" x2="{pt_sim[0]}" y2="{pt_sim[1]}" stroke="#ff0000" stroke-width="0.2" />'
        svg += f'<circle cx="{pt[0]}" cy="{pt[1]}" r="1" fill="#000000" />'
svg += '</svg>'
rt.displaySVG(svg)